In [1]:
from scipy.stats import wilcoxon
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import json

In [11]:
all= pd.read_csv('../AllWindowData.csv')
data=pd.DataFrame({'id':all["userId"],"case":all['caseNum'], 'Window Type':all['type'],'Environment': all[ "environment"],'Distance':all["distance"],'Window Anchor':all['windowMode'],'Opacity':all['transparency'],'Size':all['visual angle'],'Meeting Format':all['Meeting type'], 'Theta':all['theta'], 'Phi':all['phi'], 'Pitch':all['pitch'], 'Row':all['row'], 'Offset':all['offset'], 'x': all['x'], 'y': all['y'], 'z': all['z'], 'scale': all['scale']})
data['Window Type'] = data['Window Type'].replace({0: 'Shared Screen', 1: 'Speaker\'s Video'})
data['Window Anchor'] = data['Window Anchor'].replace({0: 'Path Anchor', 1: 'Head Anchor'})
data['Meeting Format'] = data['Meeting Format'].replace({1: 'Listening', 2: 'Discussion'})
data['Environment'] = data['Environment'].replace({'low': 'Low-Traffic', 'high': 'High-Traffic'})
data['Opacity']=1-data['Opacity']

In [12]:
def w(data, para, value):
    grouped = data.groupby(['Window Type', para]).apply(lambda x: x[[para, 'Window Type']+value])
    p = grouped[para].unique()
    title = f'{"Factor":11s}\t{"Window Type":15s}'
    for e in value:
        title += f'\t{e:>12s}'
    print(title)
    for w in ['Shared Screen', 'Speaker\'s Video']:        
        for pp in p:        
            a = grouped[(grouped['Window Type']==w) & (grouped[para]==pp)]
            mean = f'{para:11s}\t{w:15s}'
            std = f'{para:11s}\t{w:15s}'
            for e in value:
                mean += f'\t{a[e].mean():12.2f}'
                std += f'\t{a[e].std():12.2f}'
            mean += "\tMean"
            std += "\tSTD"
            print(mean)
            print(std)
    print("") 

In [13]:
w(data, 'Environment', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Opacity', 'Distance'])
w(data, 'Meeting Format', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Opacity', 'Distance'])
w(data, 'Window Anchor', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Opacity', 'Distance'])

Factor     	Window Type    	        Size	       Theta	         Phi	       Pitch	         Row	     Opacity	    Distance
Environment	Shared Screen  	        0.26	       -5.88	      -18.77	        6.92	       -0.17	        0.84	        0.86	Mean
Environment	Shared Screen  	        0.13	       15.64	       16.17	        9.19	        7.10	        0.14	        0.27	STD
Environment	Shared Screen  	        0.21	       -4.22	      -13.49	        5.47	       -0.50	        0.81	        0.89	Mean
Environment	Shared Screen  	        0.11	       16.03	       21.92	       12.37	        6.93	        0.22	        0.29	STD
Environment	Speaker's Video	        0.04	        7.50	      -16.23	        4.74	       -6.92	        0.89	        0.86	Mean
Environment	Speaker's Video	        0.02	       22.59	       14.07	       12.72	       11.14	        0.17	        0.30	STD
Environment	Speaker's Video	        0.04	        3.75	      -11.77	        5.81	       -5.91	        0.75	        0.85	Mean
Environment	Spea

In [7]:
def whp(data, para, value):
    print(f'{"Factor":11s}\t{"Window Anchor"}\t{"Window Type":15s}\t\t{"Value":12s}\t{"P-value":7s}\t{"sig":3s}\t{"mean1":7s}\t{"mean2":7s}')
    grouped = data.groupby(['Window Type', para]).apply(lambda x: x[[para, "Window Anchor", 'Window Type']+value])
    # print(grouped)
    p = grouped[para].unique()
    # All
    for anchor in ['Path Anchor', "Head Anchor"]: 
        a = grouped[(grouped[para]==p[0]) & (grouped['Window Anchor']==anchor)]
        b = grouped[(grouped[para]==p[1]) & (grouped['Window Anchor']==anchor)]
        for e in value:
            statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
            significant = getStar(p_value)
            print(f'{para:11s}\t{anchor:13s}\t{"All":15s}\t\t{e:12s}\t{p_value:.5f}\t{significant}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    #
    for anchor in ['Path Anchor', "Head Anchor"]: 
        for w in ['Shared Screen', 'Speaker\'s Video']:                       
            a = grouped[(grouped['Window Type']==w) & (grouped['Window Anchor']==anchor) & (grouped[para]==p[0])]
            b = grouped[(grouped['Window Type']==w) & (grouped['Window Anchor']==anchor) & (grouped[para]==p[1])]
            for e in value:
                statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
                significant = getStar(p_value)
                print(f'{para:11s}\t{anchor:13s}\t{w:15s}\t\t{e:12s}\t{p_value:.4f}\t{significant:3s}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    print("") 

In [8]:
whp(data, 'Environment', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Transparency', 'Distance'])
whp(data, 'Meeting Format', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Transparency', 'Distance'])

Factor     	Window Anchor	Window Type    		Value       	P-value	sig	mean1  	mean2  
Environment	Path Anchor  	All            		Size        	0.16553		0.14863	0.13342
Environment	Path Anchor  	All            		Theta       	0.48808		2.13435	-1.77294
Environment	Path Anchor  	All            		Phi         	0.00160	**	-16.05483	-8.79600
Environment	Path Anchor  	All            		Pitch       	0.07403		5.51962	3.42139
Environment	Path Anchor  	All            		Row         	0.89616		-3.63651	-3.32672
Environment	Path Anchor  	All            		Transparency	0.00272	**	0.87132	0.79579
Environment	Path Anchor  	All            		Distance    	0.26377		0.88760	0.85855
Environment	Head Anchor  	All            		Size        	0.05013		0.14633	0.12153
Environment	Head Anchor  	All            		Theta       	0.07590		-0.51103	1.31157
Environment	Head Anchor  	All            		Phi         	0.23726		-18.94667	-16.46174
Environment	Head Anchor  	All            		Pitch       	0.88023		6.13452	7.85329
Environmen

In [20]:
def wwt(data, value):
    print(f'{"Value":12s}\t{"P-value":7s}\t\t{"sig":3s}\t{"mean1":7s}\t{"mean2":7s}')
    grouped = data.groupby(['Window Type']).apply(lambda x: x[['Window Type']+value])

    # All
    a = grouped[(grouped['Window Type']=='Shared Screen')]
    b = grouped[(grouped['Window Type']=='Speaker\'s Video')]
    for e in value:
        statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
        significant = getStar(p_value)
        print(f'{e:12s}\t{p_value:.5f}\t\t{significant}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    #
    # for w in ['Shared Screen', 'Speaker\'s Video']:                
    #     a = grouped[(grouped['Window Type']==w)]
    #     b = grouped[(grouped['Window Type']==w)]
    #     for e in value:
    #         statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
    #         significant = getStar(p_value)
    #         print(f'{w:15s}\t\t{e:12s}\t{p_value:.5f}\t{significant:3s}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    print("") 

In [21]:
wwt(data, ['Size', 'Offset', 'Theta', 'Phi', 'Transparency'])

Value       	P-value		sig	mean1  	mean2  
Size        	0.00000		***	0.35421	0.19240
Offset      	0.00005		***	2.50064	2.37038
Theta       	0.00001		***	1.65888	1.47257
Phi         	0.07924			1.85226	1.81519
Transparency	0.49844			0.82223	0.81770

